<a href="https://colab.research.google.com/github/codebyshennan/nyp_applied_ai/blob/main/ITI102-DSFoundation/assignment2/ITI102_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ITI102 Assignment 2

## Q1
a) Use the python Scrapy module to implement a web scraping program to
get the content from the following websites (4 marks)

In [ ]:
!pip -q install scrapy crochet folium

In [ ]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import json
from urllib.parse import urlparse


setup()

base_url = 'https://webscraper.io/test-sites/e-commerce/allinone/computers/'
slugs = ['tablets', 'laptops']

class ProductsToJson(scrapy.Spider):
    name = "WebScrapProductsToJson"
    start_urls = [ base_url + slug for slug in slugs]
    custom_settings = {
        'FEEDS': {
            'products.json': {
                'format': 'json',
                'overwrite': True
            }
        }
    }

    """
    requirement:
    - {
        “type”: “tablet”,
        "price": ["$603.99"],
        "description": ["Wi-Fi, 64GB, Silver"],
        "product": "Apple iPad Air",
        "review": "7 reviews"
      }
    """
    def parse(self, response):
        """parse data from urls"""
        parsed_url = urlparse(response.request.url)
        slug = parsed_url.path.strip('/').split('/')[-1]

        for meta in response.css('body > div.wrapper > div.container.test-site > div.row > div.col-md-9 > div.row > div > div.thumbnail'):
          price = meta.css('div.thumbnail > div.caption > h4::text')[0].get()
          product = meta.css('div.thumbnail > div.caption > h4 > a::text')[0].get()
          description = meta.css('div.thumbnail > div.caption > p.description::text').get()
          ratings = meta.css('div.thumbnail > div.ratings > p.pull-right::text').get()

          thumbnail = {
              "type": slug,
              "price": [price],
              "description": [description],
              "review": ratings
          }

          yield thumbnail



@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(ProductsToJson)
    return d

In [ ]:
run_spider()


It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.


b) Develop a python function to search for a product information based on the
review. (2 marks)

In [ ]:
import json

def search_product(filepath, query, key):
  with open(filepath) as file:
    data = json.load(file)

  results = [item for item in data if key in item and query in item[key]]
  return results

In [ ]:
print("There are ", len(search_product('./products.json', '8', 'review')), "products with 8 reviews.")

There are  12 products with 8 reviews.


In [ ]:
print("There are ", len(search_product('./products.json', '14', 'review')), "products with 14 reviews.")

There are  11 products with 14 reviews.


## Q2:
Implement a Singapore traffic report system using python.

a) Python program request for the traffic incident using URL and get the return JSON data. Extract the traffic incident data and display in the Singapore map (3 marks)

ref: https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [ ]:
import requests

url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
headers = {
    'AccountKey': '8JUcVpAQQqiLFJMkisjndw==',
    # 'X-API-Key': 'token'
}
payload = {}

response = requests.request("GET",
                            url,
                            headers = headers,
                            data = payload
                            )

trf_data = response.json()
df_incidents = pd.json_normalize(trf_data, record_path = ['value'])
# print(json.dumps(trf_data, indent=2))
df_incidents

,Type,Latitude,Longitude,Message
0,Vehicle breakdown,1.418127,103.801726,(1/7)16:35 Vehicle breakdown on SLE (towards B...
1,Roadwork,1.315662,103.844944,(1/7)16:25 Roadworks on Thomson Road (towards ...
2,Vehicle breakdown,1.316423,103.758470,(1/7)16:18 Vehicle breakdown on AYE (towards T...
3,Heavy Traffic,1.436897,103.768490,(1/7)16:15 Heavy Traffic on BKE (towards Woodl...
4,Roadwork,1.302002,103.855579,(1/7)16:05 Roadworks on Queen Street (towards ...
...,...,...,...,...
76,Roadwork,1.330895,103.934424,(1/7)9:45 Roadworks on Bedok North Avenue 3 (t...
77,Roadwork,1.329726,103.685699,(1/7)9:30 Roadworks on Upper Jurong Road (towa...
78,Roadwork,1.365996,103.891327,(1/7)9:27 Roadworks on Upper Serangoon Road (t...
79,Roadwork,1.305307,103.910139,(1/7)9:17 Roadworks on Still Road South (towar...


In [ ]:
locations = df_incidents[['Latitude', 'Longitude']]
locList = locations.values.tolist()

In [ ]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map(location=[1.29, 103.85], zoom_start=11, tiles = "cartodb positron")

marker_cluster = MarkerCluster().add_to(map)

for point in range(0, len(locList)):
    folium.Marker(locList[point], popup=df_incidents['Message'][point]).add_to(marker_cluster)
map

b) Python program request for the traffic band using URL and get the return JSON data. Extract the traffic band data and display in the Singapore map (3 marks)

In [ ]:
url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2"
headers = {
    'AccountKey': '8JUcVpAQQqiLFJMkisjndw==',
    # 'X-API-Key': 'token'
}
payload = {}

response = requests.request("GET",
                            url,
                            headers = headers,
                            data = payload
                            )

trf_data = response.json()
df_bands = pd.json_normalize(trf_data, record_path = ['value'])
# print(json.dumps(trf_data, indent=2))
df_bands

,LinkID,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,Location
0,103000000,KENT ROAD,E,3,20,29,1.3170142376560023 103.85298052044503 1.316684...
1,103000010,BUCKLEY ROAD,E,8,70,999,1.3166507852203482 103.84102305136321 1.316912...
2,103000011,BUCKLEY ROAD,E,8,70,999,1.316912438354752 103.84022564204443 1.3166507...
3,103000014,SHREWSBURY ROAD,E,5,40,49,1.3186726294030418 103.84700267615683 1.318021...
4,103000015,SHREWSBURY ROAD,E,4,30,39,1.3180212058893457 103.84711392225329 1.318672...
...,...,...,...,...,...,...,...
495,103001218,DRAYCOTT DRIVE,D,8,70,999,1.3096279339357806 103.83110989321554 1.309694...
496,103001219,DRAYCOTT DRIVE,D,5,40,49,1.309694857000047 103.83119447352243 1.3096279...
497,103001226,PADANG JERINGAU,D,4,30,39,1.3099051585628567 103.86554520014256 1.309253...
498,103001227,PADANG JERINGAU,D,4,30,39,1.309253800591554 103.86611733337693 1.3099051...


In [ ]:
# Empty list to store parsed values
# startlon, startlat, endlon, endlat

split_values = [location_str.split() for location_str in df_bands["Location"]]
start_lat, start_long, end_lat, end_long = zip(*[(float(s[0]), float(s[1]), float(s[2]), float(s[3])) for s in split_values])


latlong_origin = list(zip(start_lat, start_long))
latlong_destination = list(zip(end_lat, end_long))

c) Use different markers to represent and differentiate the traffic incident and traffic bands in the map.Allow the markers to display more information when it require (3 marks)

In [ ]:
for index, (origin, destination) in enumerate(zip(latlong_origin, latlong_destination)):
    folium.CircleMarker([origin[0], origin[1]],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(marker_cluster)

    folium.CircleMarker([destination[0], destination[1]],
                        radius=15,
                        fill_color="red", # divvy color
                       ).add_to(marker_cluster)

# TODO: add icon
    folium.PolyLine([[origin[0], origin[1]],
                     [destination[0], destination[1]]], popup=df_bands['RoadName'][index]).add_to(marker_cluster)

map

## Q3

You are given a set of text data that expresses the sentiments of customers.

The sentiments are label as follow:
```
pos- positive
neg- negative
```

The text data are stored in the Train and Test folders with two subfolders `pos` and `neg`. In each of these, the subfolder contains 100 text reviews.

Refer to the `Q3sentimentClassificationV2_student.ipynb`.

Complete the data preprocessing tasks in the ipynb file using `python` Natural Language Toolkit( https://www.nltk.org/).

## Q4:

Given the following `dataset1.csv`

Here show the first 5 rows of a population data

| Unnamed: 0 | age | educatn | earnings | hours | kids | married  |
|------------|-----|---------|----------|-------|------|----------|
| 0          | 39  | 12.0    | 77250    | 2940  | 2    | married  |
| 1          | 35  | 12.0    | 12000    | 2040  | 2    | divorced |
| 2          | 33  | 12.0    | 8000     | 693   | 1    | married  |
| 3          | 39  | 10.0    | 15000    | 1904  | 2    | married  |
| 4          | 47  | 9.0     | 6500     | 1683  | 5    | married  |

Given the follow Hypothesis:

Null Hypothesis `HO`:

Work hours for people with higher earnings == Work hours for people with lower earnings

Alternative Hypothesis `HA`:

Work hours for people with higher earnings > Work hours for people with lower earnings

Conduct the hypothesis test with sample data using python `scipy.stats`function.

State your result of the hypothesis test.

In [ ]:
import pandas as pd
from scipy import stats

# Read the dataset
data = pd.read_csv('dataset1.csv')

# Extract the necessary columns
earnings = data['earnings']
hours = data['hours']

# Split the data into two groups based on earnings (higher and lower)
higher_earnings = hours[earnings > earnings.median()]
lower_earnings = hours[earnings <= earnings.median()]

# Perform the hypothesis test using an independent t-test
result = stats.ttest_ind(higher_earnings, lower_earnings, alternative='greater')

# Print the result
if result.pvalue < 0.05:
    print("Reject the null hypothesis. There is evidence that work hours for people with higher earnings are greater than work hours for people with lower earnings.")
else:
    print("Fail to reject the null hypothesis. There is not enough evidence to conclude that work hours for people with higher earnings are greater than work hours for people with lower earnings.")

FileNotFoundError: ignored

## Q5:
An IOT device collected data in the following format:

1. `dev-144` temperature is 40.5 degree celsius, pressure is 20000.5 pa
2. `dev-146` pressure is 24000.5 pa, temperature is 35.5 degree Celsius
3. `dev-23` temperature is 33.5 degree celsius, pressure is 10400.3 pa
4. `dev-25` pressure is 40040.5 pa, temperature is 24.5 degree Celsius

Use python regular expression to extract the data and store into a Json format.

```
{
  “device”:”dev-144”,
  “temperature”: 40.5,
  “pressure”: 20000.5
}
```

In [ ]:
import re
import json

data = [
    "dev-144 temperature is 40.5 degree celsius, pressure is 20000.5 pa",
    "dev-146 pressure is 24000.5 pa, temperature is 35.5 degree Celsius",
    "dev-23 temperature is 33.5 degree celsius, pressure is 10400.3 pa",
    "dev-25 pressure is 40040.5 pa, temperature is 24.5 degree Celsius"
]

device_pattern = r'dev-(\d+)'
temperature_pattern = r'temperature is (\d+\.\d+)'
pressure_pattern = r'pressure is (\d+\.\d+)'

result = []

for item in data:
    device_match = re.search(device_pattern, item)
    temperature_match = re.search(temperature_pattern, item, re.IGNORECASE)
    pressure_match = re.search(pressure_pattern, item, re.IGNORECASE)

    if device_match and temperature_match and pressure_match:
        device = device_match.group(1)
        temperature = float(temperature_match.group(1))
        pressure = float(pressure_match.group(1))

        result.append({
            'device': 'dev-' + device,
            'temperature': temperature,
            'pressure': pressure
        })

json_result = json.dumps(result, indent=4)

print(json_result)


[
    {
        "device": "dev-144",
        "temperature": 40.5,
        "pressure": 20000.5
    },
    {
        "device": "dev-146",
        "temperature": 35.5,
        "pressure": 24000.5
    },
    {
        "device": "dev-23",
        "temperature": 33.5,
        "pressure": 10400.3
    },
    {
        "device": "dev-25",
        "temperature": 24.5,
        "pressure": 40040.5
    }
]
